In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6" color="#009e84"> ch03. 연관분석 </font></b>

- pip install apyori

# 1. 연관분석 개요

- 데이터들 사이에 자주 발생하는 속성을 찾고, 그 속성들 사이의 연관성이 어느정도 있는지 분석
- 활용분야 : 상품진열, 사기보험적발, 신상품 카테고리 구성, ...
```
조건(left-hand side, item_base) : 오렌지주스(x)    →    결과(right-hand side, item_add) : 와인(y)

연관분석의 지표 (lecnote의 cf_basket.csv 내용)
1. 지지도(support) : 전체 데이터 중에서 조건, 결과 항목들이 포함된 거래 비율
                    (얼마나 자주 함께 나타나는지)
                    (x → y)의 항목수 / 전체 데이터수 = 0.2
            
2. 신뢰도(confidence) : 조건이 발생했을 때, 결과가 동시에 일어날 확률
                       (조건이 오면 결과가 얼마나 자주 나오는지)
                       (x → y)의 항목수 / x가 나오는 항목수 = 0.5
                       

3. 향상도(lift) : 우연히 발생할 규칙은 아니었는지 확인
                 <1 : 함께 나타날 가능성이 기대보다 낮자
                 >1 : 양의 상관관계
                 (x → y)의 지지도 / (x의 지지도 * y의 지지도) = 0.2 / (0.4 * 0.6) = 0.834


# 2. 연관분석 구현

In [3]:
import csv
transaction = []
with open('data/cf_basket.csv', 'r', encoding='utf-8') as f:
    csvdata = csv.reader(f)
    transaction = list(csvdata)
transaction

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['맥주', '콜라', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인']]

In [31]:
from apyori import apriori

rules = apriori(transactions=transaction, 
                min_support=0.15,    # min_support같은 키워드인자는 자동완성이 안됨
                min_confidence=0.1,  # 신뢰도가 0.1보다 작은것들은 넣지않게하려고 하는것
                min_lift=1.00001)    # 1보다 큰것들을 넣고싶어서. 1도 안들어가게 하려고 이렇게 적는것
rules = list(rules)
len(rules)

6

In [30]:
# 내용중 가장 복잡한 친구

row = rules[15] # {'맥주', '콜라', '와인'} 지지도 0.2
print(row[0])
row # 리스트가 아닌것처럼 생겼지만 리스트

frozenset({'맥주', '콜라', '와인'})


RelationRecord(items=frozenset({'맥주', '콜라', '와인'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'맥주', '콜라', '와인'}), confidence=0.2, lift=1.0), OrderedStatistic(items_base=frozenset({'맥주'}), items_add=frozenset({'콜라', '와인'}), confidence=0.5, lift=1.25), OrderedStatistic(items_base=frozenset({'와인'}), items_add=frozenset({'맥주', '콜라'}), confidence=0.33333333333333337, lift=0.8333333333333334), OrderedStatistic(items_base=frozenset({'콜라'}), items_add=frozenset({'맥주', '와인'}), confidence=0.25, lift=1.25), OrderedStatistic(items_base=frozenset({'맥주', '와인'}), items_add=frozenset({'콜라'}), confidence=1.0, lift=1.25), OrderedStatistic(items_base=frozenset({'맥주', '콜라'}), items_add=frozenset({'와인'}), confidence=0.5, lift=0.8333333333333334), OrderedStatistic(items_base=frozenset({'콜라', '와인'}), items_add=frozenset({'맥주'}), confidence=0.5, lift=1.25)])

In [13]:
row[2]

[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'맥주', '콜라', '와인'}), confidence=0.2, lift=1.0),
 OrderedStatistic(items_base=frozenset({'맥주'}), items_add=frozenset({'콜라', '와인'}), confidence=0.5, lift=1.25),
 OrderedStatistic(items_base=frozenset({'와인'}), items_add=frozenset({'맥주', '콜라'}), confidence=0.33333333333333337, lift=0.8333333333333334),
 OrderedStatistic(items_base=frozenset({'콜라'}), items_add=frozenset({'맥주', '와인'}), confidence=0.25, lift=1.25),
 OrderedStatistic(items_base=frozenset({'맥주', '와인'}), items_add=frozenset({'콜라'}), confidence=1.0, lift=1.25),
 OrderedStatistic(items_base=frozenset({'맥주', '콜라'}), items_add=frozenset({'와인'}), confidence=0.5, lift=0.8333333333333334),
 OrderedStatistic(items_base=frozenset({'콜라', '와인'}), items_add=frozenset({'맥주'}), confidence=0.5, lift=1.25)]

In [35]:
row = rules[5]
support = row[1]
ordered_st = row[2]
print('지지도 :', support)
# print('순서들 :', ordered_st)
for item in ordered_st:
    lhs = item[0]
    lhs = ','.join([x for x in lhs])
    rhs = item[1]
    rhs = ','.join([x for x in rhs])
    confidence = item[2]
    lift = item[3]
    print('{:8} => {:10}\t{}\t{:.2f}\t{:.2f}'.format(lhs, rhs, support, confidence, lift))

지지도 : 0.2
콜라       => 소주,와인     	0.2	0.25	1.25
소주,와인    => 콜라        	0.2	1.00	1.25


In [42]:
print('조건 → 결과 \t\t\t\t 지지도  신뢰도  향상도')
for row in rules:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = item[0]
        lhs = ','.join([x for x in lhs])
        rhs = item[1]
        rhs = ','.join([x for x in rhs])
        confidence = item[2]
        lift = item[3]
        print('{:10} => {:16}\t{}\t{:}\t{:.2f}'.format(lhs, rhs, support, round(confidence, 2), lift))

조건 → 결과 				 지지도  신뢰도  향상도
맥주         => 콜라              	0.4	1.0	1.25
콜라         => 맥주              	0.4	0.5	1.25
소주         => 콜라              	0.6	1.0	1.25
콜라         => 소주              	0.6	0.75	1.25
콜라         => 맥주,소주           	0.2	0.25	1.25
맥주,소주      => 콜라              	0.2	1.0	1.25
맥주         => 콜라,와인           	0.2	0.5	1.25
콜라         => 맥주,와인           	0.2	0.25	1.25
맥주,와인      => 콜라              	0.2	1.0	1.25
콜라,와인      => 맥주              	0.2	0.5	1.25
소주         => 오렌지주스,콜라        	0.2	0.33	1.67
콜라         => 오렌지주스,소주        	0.2	0.25	1.25
오렌지주스,소주   => 콜라              	0.2	1.0	1.25
오렌지주스,콜라   => 소주              	0.2	1.0	1.67
콜라         => 소주,와인           	0.2	0.25	1.25
소주,와인      => 콜라              	0.2	1.0	1.25


In [46]:
import pandas as pd

rules_df = pd.DataFrame(None, columns=['lhs', 'rhs', '지지도', '신뢰도', '향상도'])
rules_df

,lhs,rhs,지지도,신뢰도,향상도


In [51]:
# 데이터 한행씩 추가

# rules_df.loc[0] = ['조건','결과', 0.2, 0.9, 1.2]
# rules_df

idx = 0
for row in rules:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = item[0]
        lhs = ','.join([x for x in lhs])
        rhs = item[1]
        rhs = ','.join([x for x in rhs])
        confidence = item[2]
        lift = item[3]
        rules_df.loc[idx] = [lhs, rhs, support, round(confidence, 2), round(lift, 2)]
        idx += 1
        
rules_df.sort_values(by=['신뢰도', '향상도'], ascending=False)

,lhs,rhs,지지도,신뢰도,향상도
13,"오렌지주스,콜라",소주,0.2,1.00,1.67
0,맥주,콜라,0.4,1.00,1.25
2,소주,콜라,0.6,1.00,1.25
5,"맥주,소주",콜라,0.2,1.00,1.25
8,"맥주,와인",콜라,0.2,1.00,1.25
12,"오렌지주스,소주",콜라,0.2,1.00,1.25
15,"소주,와인",콜라,0.2,1.00,1.25
3,콜라,소주,0.6,0.75,1.25
1,콜라,맥주,0.4,0.50,1.25
6,맥주,"콜라,와인",0.2,0.50,1.25


# 3. 뉴스 연관 분석

In [53]:
test = [1,2,3,4,5]
list(filter(lambda x : x%2!=0, test))

[1, 3, 5]

In [54]:
stopwords = {'기자', '앵커'}
news_noun_list = ['기자', '앵커', '주공', '전세', '기자', '가을']
list(filter(lambda word : word not in stopwords, news_noun_list))

['주공', '전세', '가을']

In [55]:
import requests
from bs4 import BeautifulSoup
from mecab import MeCab

rss_url = 'https://fs.jtbc.co.kr/RSS/economy.xml'
jtbc_response = requests.get(rss_url)
soup = BeautifulSoup(jtbc_response.content, 'xml')
item_elems = soup.find_all('item') # item_elems = soup.select('item')

# 객체생성
mecab = MeCab()

# title, description for문
news = []
for item_elem in item_elems:
    title = item_elem.find('title').text
    description = item_elem.find('description').text
    total_text = title + ' ' + description 
    noun_list = [word for word, tag in mecab.pos(total_text) if tag in ('NNG','NNP') and word not in stopwords] # NNG, NNP(보통명사 고유명사)
    news.append(noun_list)
print(news[:3])

[['둔촌', '주공', '전세', '홍수', '예상', '눈치', '게임', '시작', '단군', '최대', '규모', '재건축', '둔촌', '주공', '아파트', '입주', '다음', '달', '시작', '규모', '입주', '전세', '매물', '현장', '분위기'], ['실제', '캠', '핑장', '관리', '소홀', '캠', '핑장', '예약', '때', '예약', '플랫폼', '사진', '실제', '불만', '경우', '앞', '예약', '플랫', '폼', '책임', '강화', '아람', '인터넷', '카페'], ['요즘', '편의점', '정도', '육박', '점심', '값', '편의점', '도시락', '때', '편의점', '도시락', '식비', '부담', '아람', '서울']]


In [ ]:
# cnt_anchor = 0 # 앵커가 나온 횟수
# cnt_editor = 0 # 기자가 나온 횟수
# for item in news:
#     for word in item:
#         if word=='기자':
#             cnt_editor += 1
#         elif word=='앵커':
#             cnt_anchor += 1
# cnt_anchor, cnt_editor

In [62]:
rules = apriori(news, min_support=0.15, min_confidence=0.1)
rules = list(rules)

rules_df = pd.DataFrame(None, columns=['조건', '결과', '지지도', '신뢰도', '향상도'])
rules_df

,조건,결과,지지도,신뢰도,향상도


In [63]:
idx = 0
for row in rules:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = item[0]
        lhs = ','.join([x for x in lhs])
        rhs = item[1]
        rhs = ','.join([x for x in rhs])
        confidence = item[2]
        lift = item[3]
        rules_df.loc[idx] = [lhs, rhs, support, round(confidence, 2), round(lift, 2)]
        idx += 1
        
rules_df.sort_values(by=['신뢰도', '향상도'], ascending=False)

,조건,결과,지지도,신뢰도,향상도
16,말,아람,0.15,0.60,2.4
17,아람,말,0.15,0.60,2.4
2,,말,0.25,0.25,1.0
6,,아람,0.25,0.25,1.0
12,,정부,0.25,0.25,1.0
4,,분기,0.20,0.20,1.0
9,,오늘,0.20,0.20,1.0
10,,올해,0.20,0.20,1.0
11,,이상,0.20,0.20,1.0
0,,가격,0.15,0.15,1.0
